In [1]:
import sys
sys.path.append('./data')

import argparse
import os
import random
import shutil
import time
import warnings
import math
from matplotlib import pyplot as plt
from dataset import flowerDataset
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.autograd import Variable
print("Pytorch Version: ", torch.__version__)

Pytorch Version:  1.7.1


In [2]:
W_BIT = 4
A_BIT = 4

def uniform_quantize(k):
  class qfn(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
      if k == 32:
        out = input
      elif k == 1:
        out = torch.sign(input)
      else:
        n = float(2 ** k  - 1)
        out = torch.round(input * n) / n
      return out

    @staticmethod
    def backward(ctx, grad_output):
      grad_input = grad_output.clone()
      return grad_input

  return qfn().apply


class weight_quantize_fn(nn.Module):
  def __init__(self, w_bit):
    super(weight_quantize_fn, self).__init__()
    assert w_bit <= 8 or w_bit == 32
    self.w_bit = w_bit
    self.uniform_q = uniform_quantize(k=w_bit - 1)

  def forward(self, x):
    # print('===================')
    if self.w_bit == 32:
      weight_q = x
    elif self.w_bit == 1:
      E = torch.mean(torch.abs(x)).detach()
      weight_q = (self.uniform_q(x / E) + 1) / 2 * E
    else:
      weight = torch.tanh(x)
      weight = weight / torch.max(torch.abs(weight))
      weight_q = self.uniform_q(weight)
    return weight_q


def conv2d_Q_fn(w_bit):
  class Conv2d_Q(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
      super(Conv2d_Q, self).__init__(in_channels, out_channels, kernel_size, stride,
                                     padding, dilation, groups, bias)
      self.w_bit = w_bit
      self.quantize_fn = weight_quantize_fn(w_bit=w_bit)

    def forward(self, input, order=None):
      weight_q = self.quantize_fn(self.weight)
      return F.conv2d(input, weight_q, self.bias, self.stride,
                      self.padding, self.dilation, self.groups)

  return Conv2d_Q

def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

class activation_quantize_fn(nn.Module):
  def __init__(self, a_bit):
    super(activation_quantize_fn, self).__init__()
    assert a_bit <= 8 or a_bit == 32
    self.a_bit = a_bit
    self.uniform_q = uniform_quantize(k=a_bit)

  def forward(self, x):
    if self.a_bit == 32:
      activation_q = x
    else:
      activation_q = self.uniform_q(torch.clamp(x, 0, 1))
    return activation_q


def conv3x3_bn_Q(inp, oup, stride, padding):
    return nn.Sequential(
        conv2d_Q_fn(W_BIT)(inp, oup, kernel_size=3, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(oup),
        activation_quantize_fn(A_BIT) #ReLU
    )

# the last conv block
def conv1x1_bn_Q(inp, oup):
    return nn.Sequential(
        conv2d_Q_fn(W_BIT)(inp, oup, kernel_size=1, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(oup),
        activation_quantize_fn(A_BIT) #ReLU
    )

class InvertedResidual_Q(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual_Q, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup

        if expand_ratio == 1: # the first bottleneck
            self.conv = nn.Sequential(
                # dw
                conv2d_Q_fn(W_BIT)(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                activation_quantize_fn(A_BIT), #ReLU
                # pw
                conv2d_Q_fn(W_BIT)(hidden_dim, oup, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                conv2d_Q_fn(W_BIT)(inp, hidden_dim, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                activation_quantize_fn(A_BIT), #ReLU
                # dw
                conv2d_Q_fn(W_BIT)(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                activation_quantize_fn(A_BIT), #ReLU
                # pw
                conv2d_Q_fn(W_BIT)(hidden_dim, oup, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)

class MobileNetV2_Q(nn.Module):
    def __init__(self, num_classes=1000, width_mult=1.):
        super(MobileNetV2_Q, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = [
            # t, c, n, s
            [1,  16, 1, 1],
            [6,  24, 2, 2],
            [6,  32, 3, 2],
            [6,  64, 4, 2],
            [6,  96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = _make_divisible(32 * width_mult, 4 if width_mult == 0.1 else 8)
        layers = [conv3x3_bn_Q(3, input_channel, 2, 1)]

        # building inverted residual blocks
        block = InvertedResidual_Q

        for t, c, n, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 4 if width_mult == 0.1 else 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)

        # building last several layers
        output_channel = _make_divisible(1280 * width_mult, 4 if width_mult == 0.1 else 8) if width_mult > 1.0 else 1280
        self.conv = conv1x1_bn_Q(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


In [3]:
#宏定义一些数据，如epoch数，batchsize等
MAX_EPOCH=250
BATCH_SIZE=3
LR=0.0001
log_interval=6
val_interval=1

In [4]:
# ============================ step 1/5 数据 ============================
split_dir=os.path.join(".","data","splitData")
train_dir=os.path.join(split_dir,"train")
valid_dir=os.path.join(split_dir,"valid")

#对训练集所需要做的预处理
train_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

#对验证集所需要做的预处理
valid_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

# 构建MyDataset实例
train_data=flowerDataset(data_dir=train_dir,transform=train_transform)
valid_data=flowerDataset(data_dir=valid_dir,transform=valid_transform)

# 构建DataLoader
# 训练集数据最好打乱
# DataLoader的实质就是把数据集加上一个索引号，再返回
train_loader=DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
valid_loader=DataLoader(dataset=valid_data,batch_size=BATCH_SIZE)

In [5]:
# ============================ step 2/5 模型 ============================
net = MobileNetV2_Q(num_classes=17, width_mult=1.)
if torch.cuda.is_available():
    net.cuda()

In [6]:
def test():
    net=MobileNetV2_Q()
    x=torch.randn(2,3,224,224)
    y=net(x)
    print(y.size())
    print(y)

test()

torch.Size([2, 1000])
tensor([[ 0.1063, -0.1091,  0.0678,  ...,  0.0661,  0.0088,  0.0955],
        [ 0.1383, -0.0808,  0.1069,  ...,  0.0698,  0.0166,  0.0952]],
       grad_fn=<AddmmBackward>)


In [7]:
# ============================ step 3/5 损失函数 ============================
criterion=nn.CrossEntropyLoss()

In [8]:
# ============================ step 4/5 优化器 ============================
optimizer=optim.Adam(net.parameters(),lr=LR, betas=(0.9, 0.99))# 选择优化器

In [9]:
# ============================ step 5/5 训练 ============================
# 记录每一次的数据，方便绘图
train_curve=list()
valid_curve=list()
net.train()
accurancy_global=0.0

for epoch in range(MAX_EPOCH):
    loss_mean=0.
    correct=0.
    total=0.
    running_loss = 0.0

    for i,data in enumerate(train_loader):
        img,label=data
        img = Variable(img)
        label = Variable(label)
        if torch.cuda.is_available():
            img=img.cuda()
            label=label.cuda()
            
        # 前向传播
        out=net(img)
        optimizer.zero_grad()  # 归0梯度
        loss=criterion(out,label)#得到损失函数

        print_loss=loss.data.item()

        loss.backward()#反向传播
        optimizer.step()#优化
        if (i+1)%log_interval==0:
            print('epoch:{},loss:{:.4f}'.format(epoch+1,loss.data.item()))
        _, predicted = torch.max(out.data, 1)
        total += label.size(0)

        correct += (predicted == label).sum()
    print("============================================")
    accurancy=correct / total
    if accurancy>accurancy_global:
        torch.save(net.state_dict(), './weights/mobilenetv2_Q_best.pt')
        print("准确率由：", accurancy_global, "上升至：", accurancy, "已更新并保存权值为weights/mobilenetv2_Q_best.pt")
        accurancy_global=accurancy
    print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, 100*accurancy))

torch.save(net.state_dict(), './weights/mobilenetv2_Q_past.pt')
print("训练完毕，权重已保存为：weights/mobilenetv2_Q_past.pt")

epoch:1,loss:2.9538
epoch:1,loss:2.8270
epoch:1,loss:2.9923
epoch:1,loss:2.9392
epoch:1,loss:2.9183
epoch:1,loss:2.8458
epoch:1,loss:2.8929
epoch:1,loss:2.7588
epoch:1,loss:2.6298
epoch:1,loss:2.7325
epoch:1,loss:2.5322
epoch:1,loss:2.7869
epoch:1,loss:2.6848
epoch:1,loss:2.7512
epoch:1,loss:2.9606
epoch:1,loss:2.8645
epoch:1,loss:2.7229
epoch:1,loss:3.0160
epoch:1,loss:2.6878
epoch:1,loss:2.7180
epoch:1,loss:2.8588
epoch:1,loss:2.7943
epoch:1,loss:2.8608
epoch:1,loss:2.8489
epoch:1,loss:2.8594
epoch:1,loss:2.7971
epoch:1,loss:2.8849
epoch:1,loss:2.8680
epoch:1,loss:2.7485
epoch:1,loss:2.7330
epoch:1,loss:2.9440
epoch:1,loss:2.9263
epoch:1,loss:2.8460
epoch:1,loss:2.8862
epoch:1,loss:2.8266
epoch:1,loss:2.8539
epoch:1,loss:2.8333
epoch:1,loss:2.7962
epoch:1,loss:2.7329
epoch:1,loss:3.0261
epoch:1,loss:2.9891
epoch:1,loss:2.8130
epoch:1,loss:2.8407
epoch:1,loss:2.7957
epoch:1,loss:2.7138
epoch:1,loss:2.7205
epoch:1,loss:2.9151
epoch:1,loss:2.8952
epoch:1,loss:2.7942
epoch:1,loss:2.8611


epoch:7,loss:2.4407
epoch:7,loss:2.8512
epoch:7,loss:2.5002
epoch:7,loss:2.2868
epoch:7,loss:2.8579
epoch:7,loss:2.6264
epoch:7,loss:2.4164
epoch:7,loss:2.2308
epoch:7,loss:2.5432
epoch:7,loss:2.0660
epoch:7,loss:2.4267
epoch:7,loss:3.6236
epoch:7,loss:3.6331
epoch:7,loss:2.9448
epoch:7,loss:2.3880
epoch:7,loss:2.9637
epoch:7,loss:2.6526
epoch:7,loss:2.5904
epoch:7,loss:2.7297
epoch:7,loss:2.4544
epoch:7,loss:2.6050
epoch:7,loss:2.1212
epoch:7,loss:2.7348
epoch:7,loss:2.9996
epoch:7,loss:2.9841
epoch:7,loss:2.9647
epoch:7,loss:3.4461
epoch:7,loss:2.5399
epoch:7,loss:2.3471
epoch:7,loss:2.6957
epoch:7,loss:2.6162
epoch:7,loss:2.7523
epoch:7,loss:3.1150
epoch:7,loss:2.2120
epoch:7,loss:2.7710
epoch:7,loss:2.3937
epoch:7,loss:2.1308
epoch:7,loss:2.5488
epoch:7,loss:2.5515
epoch:7,loss:2.5581
epoch:7,loss:2.3707
epoch:7,loss:2.6052
epoch:7,loss:2.2912
epoch:7,loss:2.9706
epoch:7,loss:2.9026
epoch:7,loss:2.8467
epoch:7,loss:2.9283
epoch:7,loss:2.7147
epoch:7,loss:3.2492
epoch:7,loss:2.1454


epoch:13,loss:2.7713
epoch:13,loss:2.7674
epoch:13,loss:2.3415
epoch:13,loss:2.9060
epoch:13,loss:2.3998
epoch:13,loss:2.3306
epoch:13,loss:2.2006
epoch:13,loss:3.0779
epoch:13,loss:2.7337
epoch:13,loss:2.6102
epoch:13,loss:2.5085
epoch:13,loss:2.7207
epoch:13,loss:2.3266
epoch:13,loss:2.4081
epoch:13,loss:1.9583
epoch:13,loss:2.3495
epoch:13,loss:1.9717
epoch:13,loss:2.4419
epoch:13,loss:2.7982
epoch:13,loss:2.7063
epoch:13,loss:2.4545
epoch:13,loss:2.2940
epoch:13,loss:2.3781
epoch:13,loss:1.6091
epoch:13,loss:2.1867
epoch:13,loss:2.5975
epoch:13,loss:3.1373
epoch:13,loss:1.9162
epoch:13,loss:2.3029
epoch:13,loss:2.5318
epoch:13,loss:3.0580
epoch:13,loss:2.7936
epoch:13,loss:2.6053
epoch:13,loss:1.7966
epoch:13,loss:2.6506
epoch:13,loss:2.5048
epoch:13,loss:2.3324
epoch:13,loss:2.1161
epoch:13,loss:2.5482
epoch:13,loss:2.6814
epoch:13,loss:2.9837
epoch:13,loss:2.7036
epoch:13,loss:1.9966
epoch:13,loss:2.1557
准确率由： tensor(0.1636) 上升至： tensor(0.1682) 已更新并保存权值为weights/mobilenetv2_Q_best

epoch:19,loss:2.6300
epoch:19,loss:2.1368
epoch:19,loss:2.0044
epoch:19,loss:2.2979
epoch:19,loss:1.8896
epoch:19,loss:1.9396
epoch:19,loss:2.2967
epoch:19,loss:3.3996
epoch:19,loss:2.4114
epoch:19,loss:2.4375
epoch:19,loss:1.9431
epoch:19,loss:2.7702
epoch:19,loss:2.7405
epoch:19,loss:2.0423
epoch:19,loss:2.0797
epoch:19,loss:2.2241
epoch:19,loss:2.7288
epoch:19,loss:3.4979
epoch:19,loss:1.9656
epoch:19,loss:2.8458
epoch:19,loss:1.5218
epoch:19,loss:1.5765
epoch:19,loss:2.7524
epoch:19,loss:2.4755
epoch:19,loss:2.3553
epoch:19,loss:1.4864
epoch:19,loss:2.4704
epoch:19,loss:2.9806
epoch:19,loss:2.8582
epoch:19,loss:2.3782
epoch:19,loss:2.1704
epoch:19,loss:2.2300
epoch:19,loss:1.6155
epoch:19,loss:2.0948
epoch:19,loss:2.4201
epoch:19,loss:2.2417
epoch:19,loss:2.5906
epoch:19,loss:2.3890
epoch:19,loss:2.7603
epoch:19,loss:2.6530
准确率由： tensor(0.1893) 上升至： tensor(0.1994) 已更新并保存权值为weights/mobilenetv2_Q_best.pt
第19个epoch的识别准确率为：19%
epoch:20,loss:2.0062
epoch:20,loss:1.8785
epoch:20,loss:2.6

epoch:25,loss:2.1623
epoch:25,loss:2.6830
epoch:25,loss:2.9452
epoch:25,loss:1.7779
epoch:25,loss:1.7075
epoch:25,loss:2.4501
epoch:25,loss:1.7580
epoch:25,loss:2.1191
epoch:25,loss:1.4961
epoch:25,loss:2.1488
epoch:25,loss:2.2115
epoch:25,loss:2.4132
epoch:25,loss:1.5466
epoch:25,loss:2.4640
epoch:25,loss:2.3377
epoch:25,loss:3.0791
epoch:25,loss:2.8468
epoch:25,loss:2.3982
epoch:25,loss:1.9153
epoch:25,loss:3.3826
epoch:25,loss:2.7625
epoch:25,loss:1.8842
epoch:25,loss:2.5523
epoch:25,loss:2.0929
epoch:25,loss:2.1767
epoch:25,loss:2.3381
epoch:25,loss:1.6561
epoch:25,loss:2.4069
epoch:25,loss:2.4628
epoch:25,loss:2.1503
epoch:25,loss:2.2205
epoch:25,loss:1.9645
epoch:25,loss:2.8401
epoch:25,loss:2.2834
epoch:25,loss:2.3493
epoch:25,loss:3.2391
epoch:25,loss:2.4877
epoch:25,loss:1.6982
epoch:25,loss:2.1594
epoch:25,loss:2.5453
epoch:25,loss:2.3980
epoch:25,loss:2.0006
epoch:25,loss:2.1514
epoch:25,loss:2.1519
epoch:25,loss:2.3051
epoch:25,loss:2.5617
epoch:25,loss:1.6150
第25个epoch的识别准

...

epoch:225,loss:0.0252
epoch:225,loss:0.0594
epoch:225,loss:0.2364
epoch:225,loss:0.0266
epoch:225,loss:0.1119
epoch:225,loss:0.0054
epoch:225,loss:0.0429
epoch:225,loss:0.0282
epoch:225,loss:0.0127
epoch:225,loss:0.1659
epoch:225,loss:0.0393
epoch:225,loss:2.0551
epoch:225,loss:0.5478
epoch:225,loss:0.0191
第225个epoch的识别准确率为：96%
epoch:226,loss:0.0428
epoch:226,loss:0.0063
epoch:226,loss:0.0493
epoch:226,loss:0.1524
epoch:226,loss:0.0301
epoch:226,loss:0.0417
epoch:226,loss:0.1627
epoch:226,loss:0.0118
epoch:226,loss:0.0074
epoch:226,loss:0.0057
epoch:226,loss:0.6152
epoch:226,loss:0.3537
epoch:226,loss:0.5607
epoch:226,loss:1.0741
epoch:226,loss:0.0954
epoch:226,loss:0.1055
epoch:226,loss:0.4798
epoch:226,loss:0.0213
epoch:226,loss:0.4276
epoch:226,loss:0.4413
epoch:226,loss:0.0995
epoch:226,loss:0.7658
epoch:226,loss:0.0942
epoch:226,loss:0.1103
epoch:226,loss:0.3049
epoch:226,loss:0.0791
epoch:226,loss:0.0450
epoch:226,loss:0.4925
epoch:226,loss:0.0714
epoch:226,loss:0.0622
epoch:226,

epoch:231,loss:0.0232
epoch:231,loss:0.0024
epoch:231,loss:0.0075
epoch:231,loss:0.0132
epoch:231,loss:0.6186
epoch:231,loss:0.3032
epoch:231,loss:0.0646
epoch:231,loss:0.0325
epoch:231,loss:0.1415
epoch:231,loss:0.0442
epoch:231,loss:0.0147
epoch:231,loss:1.1966
epoch:231,loss:0.0850
epoch:231,loss:0.0009
epoch:231,loss:0.0787
epoch:231,loss:0.0085
epoch:231,loss:0.0492
epoch:231,loss:0.0159
epoch:231,loss:0.0098
第231个epoch的识别准确率为：95%
epoch:232,loss:0.0091
epoch:232,loss:0.3946
epoch:232,loss:0.1788
epoch:232,loss:0.5174
epoch:232,loss:0.0093
epoch:232,loss:0.2846
epoch:232,loss:0.0410
epoch:232,loss:0.1073
epoch:232,loss:0.6373
epoch:232,loss:0.0395
epoch:232,loss:0.1389
epoch:232,loss:0.0514
epoch:232,loss:0.0195
epoch:232,loss:0.0982
epoch:232,loss:0.0292
epoch:232,loss:0.0159
epoch:232,loss:0.0153
epoch:232,loss:0.6668
epoch:232,loss:0.1544
epoch:232,loss:0.0309
epoch:232,loss:1.1421
epoch:232,loss:0.0008
epoch:232,loss:0.5720
epoch:232,loss:0.0963
epoch:232,loss:0.0316
epoch:232,

epoch:237,loss:0.0925
epoch:237,loss:0.0048
epoch:237,loss:0.8144
epoch:237,loss:0.0456
epoch:237,loss:0.3747
epoch:237,loss:0.0145
epoch:237,loss:0.0457
epoch:237,loss:0.0054
epoch:237,loss:0.0372
epoch:237,loss:0.0120
epoch:237,loss:0.0872
epoch:237,loss:0.2158
epoch:237,loss:0.0049
epoch:237,loss:0.0147
epoch:237,loss:0.1771
epoch:237,loss:0.0128
epoch:237,loss:0.0735
epoch:237,loss:0.0323
epoch:237,loss:0.0078
epoch:237,loss:0.1396
epoch:237,loss:0.0044
epoch:237,loss:0.0158
epoch:237,loss:0.5840
epoch:237,loss:0.0153
epoch:237,loss:0.0517
epoch:237,loss:0.0101
epoch:237,loss:0.4575
epoch:237,loss:0.4098
第237个epoch的识别准确率为：95%
epoch:238,loss:0.1120
epoch:238,loss:0.2688
epoch:238,loss:0.0337
epoch:238,loss:0.0223
epoch:238,loss:0.1975
epoch:238,loss:0.0476
epoch:238,loss:0.0266
epoch:238,loss:0.0332
epoch:238,loss:0.0372
epoch:238,loss:0.0042
epoch:238,loss:0.0114
epoch:238,loss:0.0079
epoch:238,loss:0.1288
epoch:238,loss:0.0063
epoch:238,loss:0.0006
epoch:238,loss:0.0206
epoch:238,

epoch:243,loss:0.0344
epoch:243,loss:0.1924
epoch:243,loss:0.0008
epoch:243,loss:0.0090
epoch:243,loss:0.0976
epoch:243,loss:0.0752
epoch:243,loss:0.1632
epoch:243,loss:0.0457
epoch:243,loss:0.0084
epoch:243,loss:0.0212
epoch:243,loss:0.0801
epoch:243,loss:0.0268
epoch:243,loss:0.0405
epoch:243,loss:0.0505
epoch:243,loss:0.0243
epoch:243,loss:0.0725
epoch:243,loss:0.0130
epoch:243,loss:0.0095
epoch:243,loss:0.0213
epoch:243,loss:0.0932
epoch:243,loss:0.0153
epoch:243,loss:0.0048
epoch:243,loss:0.0245
epoch:243,loss:0.3335
epoch:243,loss:0.0115
epoch:243,loss:0.0416
epoch:243,loss:0.3614
epoch:243,loss:0.0580
epoch:243,loss:0.0635
epoch:243,loss:0.0077
epoch:243,loss:0.2488
epoch:243,loss:0.0174
epoch:243,loss:0.2313
epoch:243,loss:0.1402
epoch:243,loss:0.0614
epoch:243,loss:0.0108
epoch:243,loss:0.0537
第243个epoch的识别准确率为：95%
epoch:244,loss:0.0566
epoch:244,loss:0.1210
epoch:244,loss:0.0693
epoch:244,loss:0.1444
epoch:244,loss:0.0736
epoch:244,loss:0.3159
epoch:244,loss:0.0488
epoch:244,

epoch:249,loss:0.1227
epoch:249,loss:0.0631
epoch:249,loss:0.0136
epoch:249,loss:0.0213
epoch:249,loss:0.0150
epoch:249,loss:0.0433
epoch:249,loss:0.0753
epoch:249,loss:0.0742
epoch:249,loss:0.0107
epoch:249,loss:0.0189
epoch:249,loss:0.0263
epoch:249,loss:0.0298
epoch:249,loss:0.2100
epoch:249,loss:0.0126
epoch:249,loss:0.0126
epoch:249,loss:0.0982
epoch:249,loss:0.0063
epoch:249,loss:0.0078
epoch:249,loss:0.0055
epoch:249,loss:0.0771
epoch:249,loss:0.0448
epoch:249,loss:0.1718
epoch:249,loss:0.4298
epoch:249,loss:0.1393
epoch:249,loss:0.1874
epoch:249,loss:0.0189
epoch:249,loss:0.0019
epoch:249,loss:0.0008
epoch:249,loss:0.1162
epoch:249,loss:0.0368
epoch:249,loss:0.0065
epoch:249,loss:0.0585
epoch:249,loss:0.0151
epoch:249,loss:0.0057
epoch:249,loss:0.1973
epoch:249,loss:0.0185
epoch:249,loss:0.0329
epoch:249,loss:0.5411
epoch:249,loss:0.2545
epoch:249,loss:0.0261
epoch:249,loss:0.0849
epoch:249,loss:0.1491
第249个epoch的识别准确率为：95%
epoch:250,loss:0.0083
epoch:250,loss:0.0078
epoch:250,

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

split_dir=os.path.join(".","data","splitData")
test_dir=os.path.join(split_dir,"test")
# test_dir = os.path.join(".", "data", "rawData")

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_data = flowerDataset(data_dir=test_dir, transform=test_transform)
test_dataLoader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [16]:
net.load_state_dict(torch.load("./weights/mobilenetv2_Q_best.pt"))
net.eval()
correct = 0.
total=0.
with torch.no_grad():
    for i, data in enumerate(test_dataLoader):
        img, label = data
        img = Variable(img)
        label = Variable(label)
        img, label = img.to(device), label.to(device)
        print("label:", label)
        output = net(img)
        _, predicted = torch.max(output.data, 1)
        print("predict:", predicted)
        total += label.size(0)
        correct += (predicted == label).sum()

acc = correct/total * 100.
print("Total:", total)
print("Accuracy:{}%".format(acc))

label: tensor([15, 15, 15])
predict: tensor([ 4, 10, 15])
label: tensor([15, 15, 15])
predict: tensor([15, 15, 15])
label: tensor([15, 15,  9])
predict: tensor([10, 15,  9])
label: tensor([9, 9, 9])
predict: tensor([ 9,  9, 12])
label: tensor([9, 9, 9])
predict: tensor([9, 9, 9])
label: tensor([9, 2, 2])
predict: tensor([2, 2, 2])
label: tensor([2, 2, 2])
predict: tensor([ 2, 15,  2])
label: tensor([2, 2, 2])
predict: tensor([2, 2, 2])
label: tensor([4, 4, 4])
predict: tensor([16,  5,  4])
label: tensor([4, 4, 4])
predict: tensor([2, 4, 1])
label: tensor([4, 4, 0])
predict: tensor([4, 1, 0])
label: tensor([0, 0, 0])
predict: tensor([0, 0, 0])
label: tensor([0, 0, 0])
predict: tensor([ 0, 11,  0])
label: tensor([ 0, 11, 11])
predict: tensor([ 0, 12, 12])
label: tensor([11, 11, 11])
predict: tensor([11, 11, 11])
label: tensor([11, 11, 11])
predict: tensor([14, 12, 11])
label: tensor([8, 8, 8])
predict: tensor([8, 8, 8])
label: tensor([8, 8, 8])
predict: tensor([8, 6, 6])
label: tensor([8

### export

In [3]:
def generate_config(model, in_shape):
    feature_map_shape = in_shape
    print(in_shape)
    dic = {}
    conv_cnt = 0
    pool_cnt = 0
    linear_cnt = 0
    # cnt = 0
    for sub_module in model.modules():
        if type(sub_module).__base__ is torch.nn.Conv2d:
            conv_cur = {}
            conv_cur['in_shape'] = feature_map_shape[:]         
            feature_map_shape[0] = sub_module.out_channels
            feature_map_shape[1] = (feature_map_shape[1] + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            feature_map_shape[2] = (feature_map_shape[2] + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            conv_cur['out_shape'] = feature_map_shape[:]
            conv_cur['k'] = sub_module.kernel_size[0]
            conv_cur['s'] = sub_module.stride[0]
            conv_cur['p'] = sub_module.padding[0]
            
            dic['conv_' + str(conv_cnt)] = conv_cur
            
            conv_cnt = conv_cnt + 1
            # cnt = cnt + 1

        elif type(sub_module) is torch.nn.MaxPool2d:
            pool_cur = {}
            pool_cur['in_shape'] = feature_map_shape[:]
            pool_cur['p'] =  sub_module.kernel_size

            feature_map_shape[1] = feature_map_shape[1] // sub_module.kernel_size
            feature_map_shape[2] = feature_map_shape[2] // sub_module.kernel_size

            pool_cur['out_shape'] = feature_map_shape[:]

            dic['pool_' + str(pool_cnt)] = pool_cur

            pool_cnt = pool_cnt + 1
            # cnt = cnt + 1
        elif type(sub_module) is torch.nn.Linear:
            linear_cur = {}
            linear_cur['in_len'] = sub_module.in_features
            linear_cur['out_len'] = sub_module.out_features

            dic['linear_' + str(linear_cnt)] = linear_cur
            linear_cnt = linear_cnt + 1
            # cnt = cnt + 1
    
    return dic
    

def generate_params(model):
    dic = {}
    cnt = 0
    for sub_module in model.modules():
        if type(sub_module).__base__ is torch.nn.Conv2d:
            w = sub_module.weight.detach().numpy()
            dic['arr_' + str(cnt)] = w
            cnt = cnt + 1
            if sub_module.bias is not None:
                w = sub_module.bias.detach().numpy()
                dic['arr_' + str(cnt)] = w
                cnt = cnt + 1
        elif type(sub_module) is torch.nn.Linear:
            w = sub_module.weight.detach().numpy()
            dic['arr_' + str(cnt)] = w
            cnt = cnt + 1
        elif type(sub_module) is torch.nn.BatchNorm2d or type(sub_module) is torch.nn.BatchNorm1d:
            gamma = sub_module.weight.detach().numpy()
            dic['arr_' + str(cnt)] = gamma
            cnt = cnt + 1
            beta = sub_module.bias.detach().numpy()
            dic['arr_' + str(cnt)] = beta
            cnt = cnt + 1
            mean = sub_module.running_mean.numpy()
            dic['arr_' + str(cnt)] = mean
            cnt = cnt + 1
            var = sub_module.running_var.numpy()
            dic['arr_' + str(cnt)] = var
            cnt = cnt + 1
            eps = sub_module.eps
            dic['arr_' + str(cnt)] = eps
            cnt = cnt + 1
    return dic

In [4]:
model = MobileNetV2_Q(num_classes=17, width_mult=1.)
model.load_state_dict(torch.load("./weights/mobilenetv2_Q_best.pt"))
print(model)

MobileNetV2_Q(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d_Q(
        3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (quantize_fn): weight_quantize_fn()
      )
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): activation_quantize_fn()
    )
    (1): InvertedResidual_Q(
      (conv): Sequential(
        (0): Conv2d_Q(
          32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False
          (quantize_fn): weight_quantize_fn()
        )
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): activation_quantize_fn()
        (3): Conv2d_Q(
          32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
          (quantize_fn): weight_quantize_fn()
        )
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual_Q(
      (conv): Sequen

In [5]:
dic = generate_params(model)
np.savez('mobilenetv2_Q_4w4a.npz', **dic)

In [11]:
dic = generate_config(model, [3, 224, 224])
print(dic)

[3, 224, 224]
{'conv_0': {'in_shape': [3, 224, 224], 'out_shape': [32, 112, 112], 'k': 3, 's': 2, 'p': 1}, 'conv_1': {'in_shape': [32, 112, 112], 'out_shape': [32, 112, 112], 'k': 3, 's': 1, 'p': 1}, 'conv_2': {'in_shape': [32, 112, 112], 'out_shape': [16, 112, 112], 'k': 1, 's': 1, 'p': 0}, 'conv_3': {'in_shape': [16, 112, 112], 'out_shape': [96, 112, 112], 'k': 1, 's': 1, 'p': 0}, 'conv_4': {'in_shape': [96, 112, 112], 'out_shape': [96, 56, 56], 'k': 3, 's': 2, 'p': 1}, 'conv_5': {'in_shape': [96, 56, 56], 'out_shape': [24, 56, 56], 'k': 1, 's': 1, 'p': 0}, 'conv_6': {'in_shape': [24, 56, 56], 'out_shape': [144, 56, 56], 'k': 1, 's': 1, 'p': 0}, 'conv_7': {'in_shape': [144, 56, 56], 'out_shape': [144, 56, 56], 'k': 3, 's': 1, 'p': 1}, 'conv_8': {'in_shape': [144, 56, 56], 'out_shape': [24, 56, 56], 'k': 1, 's': 1, 'p': 0}, 'conv_9': {'in_shape': [24, 56, 56], 'out_shape': [144, 56, 56], 'k': 1, 's': 1, 'p': 0}, 'conv_10': {'in_shape': [144, 56, 56], 'out_shape': [144, 28, 28], 'k': 3

In [12]:
import json
#indent参数根据数据格式缩进显示，读起来更加清晰, indent的值，代表缩进空格式
json_str = json.dumps(dic, indent=4)
with open('config.json', 'w') as json_file:
    json_file.write(json_str)

#### read param

In [14]:
from Quantization import quantization

# 读出参数
# 然后根据量化规则将其处理量化为指定位宽
# 使用int32类型储存
class QNNParamReader:
    def __init__(self, paramFile):
        self.param_dict = np.load(paramFile)
        self.current_param_cnt = 0
    
    def get_last(self):
        ret = self.param_dict["arr_" + str(self.current_param_cnt)]
        # self.current_param_cnt += 1
        return ret

    def __get_current(self):
        ret = self.param_dict["arr_" + str(self.current_param_cnt)]
        self.current_param_cnt += 1
        return ret

    def read_conv_raw(self):
        w = self.__get_current()
        return w

    def read_linear_raw(self):
        w = self.__get_current()
        return w

    def read_batch_norm_raw(self):
        gamma = self.__get_current()
        beta = self.__get_current()
        mean = self.__get_current()
        var = self.__get_current()
        eps = self.__get_current()
        return (gamma, beta, mean, var, eps)

    # 读量化后卷积层参数
    # 量化后用 int32 表示每个数据
    # 默认将卷积层位宽参数量化到2个bit
    # 符号位占用一个bit
    def read_qconv_weight(self, w_bit=2):
        w = self.read_conv_raw()
        # 执行 w 量化
        qw = quantization.weight_quantize_int(w, w_bit)
        return qw

    # 读量化后的全连接层的参数
    # 量化后用 int32 表示每个数据 实际有效的只有 w_bit
    # 符号位占用一个 bit
    def read_qlinear_weight(self, w_bit=2):
        w = self.read_linear_raw()
        qw = quantization.weight_quantize_int(w, w_bit)
        return qw

    # 读取量化后的 bn 层参数
    # 将bn层和act层放在一起处理，量化后其可以用一个等差数列表示
    # 其中inc表示公差， bias表示初始值
    def read_qbarch_norm_act_param(self, w_bit=2, in_bit=4, out_bit=4, l_shift=4):
        gamma, beta, mean, var, eps = self.read_batch_norm_raw()
        qinc, qbias = quantization.bn_act_quantize_int(gamma, beta, mean, var, eps, w_bit=w_bit, in_bit=in_bit, out_bit=out_bit, l_shift=l_shift)
        return qinc, qbias

In [15]:
# 将数组中元素拼接组合
# 例如 输入 [1, 1, 1] elem_bit = 1, 返回 111
# 返回值是 int 类型 其程度可能超过 64位
def array_to_string(array, elem_bit):
        val = 0	
        #for i in range(len(array)-1, -1, -1):
        for i in range(len(array)):
            tmp = array[i]
            tmp2 = tmp

            if tmp < 0:
                tmp2 = 2**(elem_bit) + tmp 

            tmp2 = int(tmp2)
            tmp3 = tmp2 * 2**(elem_bit*i)
            val = val + tmp3
        return val

# 处理一层的参数
#  处理得到 转化后的w矩阵和 bn的inc，bias
# w矩阵为二维矩阵，row为输出通道数
# 相对于原始w张量，其内存排列顺序转换为 out_ch, row, col, in_ch
# class ParamProcess:
#     def __init__(self, w, inc, bias, w_bit, in_bit, out_bit, l_shift):
#         # self.qnn_read = QNNParamReader(file_name)
#         self.w = w
#         self.inc = inc
#         self.bias = bias
#         self.w_bit = w_bit
#         self.in_bit = in_bit
#         self.out_bit = out_bit
#         self.l_shift = l_shift

#     def conv_process(self):
#         # con_w 是一个4维张量
#         # 将输入通道维度放到最后
#         con_w = self.w
#         con_w.transpose(0, 2, 3, 1)
#         # 处理为二维矩阵
#         con_w = con_w.reshape(con_w.shape[0], -1)

#         # qinc, qbias 当前不需要处理
#         return con_w, self.inc, self.bias
    
#     def linear_process(self, w_bit, in_bit, out_bit, l_shift):
#         # linear_w0 是一个二维矩阵不需要处理
#         linear_w0 = self.qnn_read.read_qlinear_weight(w_bit)
#         linear_bn0_inc, linear_bn0_bias = self.qnn_read.read_qbarch_norm_act_param(w_bit, in_bit, out_bit, l_shift)

#         return linear_w0, linear_bn0_inc, linear_bn0_bias

#     def last_linear_process(self, w_bit):
#         # 全连接层
#         linear_w0 = self.qnn_read.read_qlinear_weight(w_bit=2)
#         return linear_w0

# 处理一层参数
# 这里的一层指的是 将conv 与 bn act 合并为一层
# 将参数整理成满足 硬件设计需求的形式
class QNNLayerMemProcess:
    # 处理 中间层用
    def __init__(self, name, reader, config, w_bit, in_bit, out_bit, l_shift, pe, simd, conv_linear=False):
        
        self.name = name
        self.reader = reader
        self.w_bit = w_bit
        self.in_bit = in_bit
        self.out_bit = out_bit
        self.l_shift = l_shift
        self.pe = pe
        self.simd = simd
        self.config = config[name] 
        self.conv_linear = conv_linear
    
    # 将矩阵整理成所需要的储存样式
    # 转化位 pe * tiles 矩阵
    def w_to_hls_array(self, w):
        # print("w shape :", w.shape)
        assert w.shape[0] % self.pe == 0, 'out_ch mod pe must 0'
        # w 矩阵的宽 其值 为 k * k * in_ch
        h = w.shape[1]
        # res0 size = out_ch, k * k * in_ch // simd + (0 or 1)
        res0 = [[0 for i in range(h // self.simd)] for j in range(w.shape[0])]
        for out_ch in range(w.shape[0]):
            for i in range(h // self.simd):
                arr = w[out_ch][i*self.simd:(i+1)*self.simd]
                res0[out_ch][i] = array_to_string(arr, self.w_bit)
            
        # 处理不够整除的部分
        if h % self.simd != 0:
            print('h mod simd != 0')
            for out_ch in range(w.shape[0]):
                arr = w[out_ch][h // self.simd * self.simd:]
                res0[out_ch].append(array_to_string(arr, self.w_bit))

        # print('res0 = ', len(res0), len(res0[0]))
        # print(np.array(res0))
        
        tiles = len(res0[0]) * (len(res0) // self.pe) 
        self.w_tiles = tiles
        # print('tiles', tiles)
        res = [[0 for i in range(tiles)] for i in range(self.pe)]

        tiles_cnt = 0
        for i in range(len(res0) // self.pe):
            for j in range(len(res0[0])):

                for pe_cnt in range(self.pe):
                    res[pe_cnt][tiles_cnt] = res0[i * self.pe + pe_cnt][j]
                tiles_cnt += 1  
        return res

    # 处理 inc 和 bias
    def inc_bias_to_hls_array(self, inc, bias):
        inc = inc.reshape(-1, self.pe)
        inc = inc.T
        bias = bias.reshape(-1, self.pe)
        bias = bias.T
        self.a_tiles = inc.shape[1]
        
        return inc, bias
    
    # 卷积参数整理
    # 返回的w因为元素可能大于64位 所以用list储存
    # inc, bias 是numpy.array类型
    def conv(self):
        w = self.reader.read_qconv_weight(self.w_bit)
        inc, bias = self.reader.read_qbarch_norm_act_param(w_bit=self.w_bit, in_bit=self.in_bit, out_bit=self.out_bit, l_shift=self.l_shift)
        # w 是二维矩阵形式
        con_w = w.transpose(0, 2, 3, 1)
        # 处理为二维矩阵
        con_w = con_w.reshape(con_w.shape[0], -1)
        # print(w)
        # 先把 w 处理为每个元素位宽都是 simd * w_bit 形式
        con_w = self.w_to_hls_array(con_w)

        inc, bias = self.inc_bias_to_hls_array(inc, bias)

        self.hls_w = con_w
        self.hls_inc = inc
        self.hls_bias = bias

        self.inc_bit_width = self.get_inc_bit_width(inc)
        self.bias_bit_width = self.get_bias_bit_width(bias)
        return con_w, inc, bias
    
    def last_conv(self):
        w = self.reader.read_qconv_weight(self.w_bit)
        # inc, bias = self.reader.read_qbarch_norm_act_param(w_bit=self.w_bit, in_bit=self.in_bit, out_bit=self.out_bit, l_shift=self.l_shift)
        # w 是二维矩阵形式
        con_w = w.transpose(0, 2, 3, 1)
        # 处理为二维矩阵
        con_w = con_w.reshape(con_w.shape[0], -1)
        # print(w)
        # 先把 w 处理为每个元素位宽都是 simd * w_bit 形式
        con_w = self.w_to_hls_array(con_w)

        # inc, bias = self.inc_bias_to_hls_array(inc, bias)

        self.hls_w = con_w
        # self.hls_inc = inc
        # self.hls_bias = bias

        # self.inc_bit_width = self.get_inc_bit_width(inc)
        # self.bias_bit_width = self.get_bias_bit_width(bias)
        return con_w #, inc, bias


    def linear(self):
        w = self.reader.read_qlinear_weight(self.w_bit)
        inc, bias = self.reader.read_qbarch_norm_act_param(w_bit=self.w_bit, in_bit=self.in_bit, out_bit=self.out_bit, l_shift=self.l_shift)

        # w = self.
        # m * n
        # 如果上一层是卷积层 需要调整参数位置
        if(self.conv_linear == True):
            last_conv_shape = self.config["last_layer_shape"]
            w = w.reshape(w.shape[0], last_conv_shape[0], last_conv_shape[1], last_conv_shape[2])
            w = w.transpose(0, 2, 3, 1)
            w = w.reshape(w.shape[0], -1)
        w = self.w_to_hls_array(w)
        inc, bias = self.inc_bias_to_hls_array(inc, bias)

        self.hls_w = w
        self.hls_inc = inc
        self.hls_bias = bias

        self.inc_bit_width = self.get_inc_bit_width(inc)
        self.bias_bit_width = self.get_bias_bit_width(bias)
        return w, inc, bias
    

    # 最后一个全连接层
    def last_linear(self):
        w = self.reader.read_qlinear_weight(self.w_bit)
        w = self.w_to_hls_array(w)
        self.hls_w = w
        return w
    
    def w_to_hls_init_str(self, w) -> str:
        w_mem_type = "const ap_uint<"+str(self.w_bit * self.simd)+">"

        res = '// ' + self.name + '_w\n'
        res += "//PEs = %d, SIMD = %d\n" % (self.pe, self.simd)
        res += '//bit = %d\n' % self.w_bit
        res += w_mem_type
        res += (' ' + self.name + '_w') 
        res += '[%d][%d] = {\n' % (len(w), len(w[0]))

        res += ",\n".join(map(lambda pe:"{\""+("\", \"".join(map(hex, pe)))+"\"}", w))
        res += '};\n'

        return res
    

    # 确定 inc 位宽 
    def get_inc_bit_width(self, inc):
        abs_max = np.abs(inc).max()
        bit_width = len(str(bin(abs_max))) - 2
        return bit_width + 1
    
    # 确定bias的位宽
    # bias 有整数和负数
    # 当前算法得出的还不是最优
    def get_bias_bit_width(self, bias):
        abs_max = np.abs(bias).max()
        bit_width = len(str(bin(abs_max))) - 2
        return bit_width + 1
    
    def inc_to_hls_init_str(self, inc) -> str:
        inc_bit_width = self.inc_bit_width

        w_mem_type = "const ap_int<"+str(inc_bit_width)+">"

        res = '// inc\n'
        res += '// ' + self.name + '_inc\n'
        res += '// w_bit = %d\n' % inc_bit_width
        res += w_mem_type
        res += (' ' + self.name + '_inc') 
        res += '[%d][%d] = {\n' % (len(inc), len(inc[0]))

        res += ",\n".join(map(lambda pe:"{\""+("\", \"".join(map(hex, pe)))+"\"}", inc))
        res += '};\n'

        return res  
    
    def bias_to_hls_init_str(self, bias) -> str:
        bias_bit_width = self.bias_bit_width

        w_mem_type = "const ap_int<"+str(bias_bit_width)+">"
        res = '// bias\n'
        res += '// ' + self.name + '_bias\n'
        res += '// w_bit = %d\n' % bias_bit_width
        res += w_mem_type
        res += (' ' + self.name + '_bias') 
        res += '[%d][%d] = {\n' % (len(bias), len(bias[0]))

        res += ",\n".join(map(lambda pe:"{\""+("\", \"".join(map(hex, pe)))+"\"}", bias))
        res += '};\n'

        return res

    def layer_param_to_init_str(self, w, inc, bias) -> str:
        res = self.w_to_hls_init_str(w)
        res += self.inc_to_hls_init_str(inc)
        res += self.bias_to_hls_init_str(bias)

        return res
    
    def last_layer_param_to_init_str(self, w) -> str:
        res = self.w_to_hls_init_str(w)
       
        return res

    def add_a_config_str(self, config_name, value) -> str:
        res = '#define %s_%s %d \n' % (self.name.upper(), config_name.upper(), value)
        return res

    def conv_config_str(self) -> str:
        res = '// ' + self.name + '\n'
        res += self.add_a_config_str('K', self.config['k'])
        res += self.add_a_config_str('S', self.config['s'])
        res += self.add_a_config_str('P', self.config['p'])
        res += self.add_a_config_str('IFM_CH', self.config['in_shape'][0])
        res += self.add_a_config_str('IFM_ROW', self.config['in_shape'][1])
        res += self.add_a_config_str('IFM_COL', self.config['in_shape'][2])

        res += self.add_a_config_str('OFM_CH', self.config['out_shape'][0])
        res += self.add_a_config_str('OFM_ROW', self.config['out_shape'][1])
        res += self.add_a_config_str('OFM_COL', self.config['out_shape'][2])

        res += self.add_a_config_str('SIMD', self.simd)
        res += self.add_a_config_str('PE', self.pe)

        res += self.add_a_config_str('IN_BIT', self.in_bit)
        res += self.add_a_config_str('OUT_BIT', self.out_bit)
        res += self.add_a_config_str('W_BIT', self.w_bit)
        res += self.add_a_config_str('INC_BIT', self.inc_bit_width)
        res += self.add_a_config_str('BIAS_BIT', self.bias_bit_width)

        res += self.add_a_config_str('W_TILES', self.w_tiles)
        res += self.add_a_config_str('A_TILES', self.a_tiles)
        res += self.add_a_config_str('L_SHIFT', self.l_shift)

        res += '\n'

        return res

    def last_conv_config_str(self) -> str:
        res = '// ' + self.name + '\n'
        res += self.add_a_config_str('K', self.config['k'])
        res += self.add_a_config_str('S', self.config['s'])
        res += self.add_a_config_str('P', self.config['p'])
        res += self.add_a_config_str('IFM_CH', self.config['in_shape'][0])
        res += self.add_a_config_str('IFM_ROW', self.config['in_shape'][1])
        res += self.add_a_config_str('IFM_COL', self.config['in_shape'][2])

        res += self.add_a_config_str('OFM_CH', self.config['out_shape'][0])
        res += self.add_a_config_str('OFM_ROW', self.config['out_shape'][1])
        res += self.add_a_config_str('OFM_COL', self.config['out_shape'][2])

        res += self.add_a_config_str('SIMD', self.simd)
        res += self.add_a_config_str('PE', self.pe)

        res += self.add_a_config_str('IN_BIT', self.in_bit)
        # res += self.add_a_config_str('OUT_BIT', self.out_bit)
        res += self.add_a_config_str('W_BIT', self.w_bit)
        # res += self.add_a_config_str('INC_BIT', self.inc_bit_width)
        # res += self.add_a_config_str('BIAS_BIT', self.bias_bit_width)

        res += self.add_a_config_str('W_TILES', self.w_tiles)
        res += self.add_a_config_str('L_SHIFT', self.l_shift)
        # res += self.add_a_config_str('A_TILES', self.a_tiles)

        res += '\n'

        return res

    def linear_config_str(self) -> str:
        res = '// ' + self.name + '\n'

        res += self.add_a_config_str('IN_LEN', self.config['in_len'])
        res += self.add_a_config_str('OUT_LEN', self.config['out_len'])

        res += self.add_a_config_str('SIMD', self.simd)
        res += self.add_a_config_str('PE', self.pe)

        res += self.add_a_config_str('IN_BIT', self.in_bit)
        res += self.add_a_config_str('OUT_BIT', self.out_bit)
        res += self.add_a_config_str('W_BIT', self.w_bit)
        res += self.add_a_config_str('INC_BIT', self.inc_bit_width)
        res += self.add_a_config_str('BIAS_BIT', self.bias_bit_width)

        res += self.add_a_config_str('W_TILES', self.w_tiles)
        res += self.add_a_config_str('A_TILES', self.a_tiles)
        res += self.add_a_config_str('L_SHIFT', self.l_shift)

        res += '\n'
        return res

    def last_linear_config_str(self) -> str:
        res = '// ' + self.name + '\n'

        res += self.add_a_config_str('IN_LEN', self.config['in_len'])
        res += self.add_a_config_str('OUT_LEN', self.config['out_len'])

        res += self.add_a_config_str('SIMD', self.simd)
        res += self.add_a_config_str('PE', self.pe)

        res += self.add_a_config_str('IN_BIT', self.in_bit)
        # res += self.add_a_config_str('OUT_BIT', self.out_bit)
        res += self.add_a_config_str('W_BIT', self.w_bit)

        res += self.add_a_config_str('L_SHIFT', self.l_shift)

        return res

In [16]:
# conv       0      1   2       3   4   5
w_bit   =   [4,     4,  4,      4,  4,   8]
in_bit  =   [8,     4,  4,      4,  4,   4]
out_bit =   [4,     4,  4,      4,  4,  32]
l_shift =   [6,     6,  6,      6,  6,   6]
simd    =   [3,     16,  16,    16, 8,   8]
pe      =   [16,    8,   8,     4,  2,   2]

In [26]:
w_bit = [4]
for i in range(50):
    w_bit.append(4)
w_bit.append(8)

in_bit = [8]
for i in range(51):
    in_bit.append(4)
    
out_bit = [4]
for i in range(50):
    out_bit.append(4)
out_bit.append(32)

l_shift = [6]
for i in range(51):
    l_shift.append(6)
    
simd = [3]
for i in range(51):
    simd.append(4)
    
pe = [4]
for i in range(51):
    pe.append(4)
    
print(len(w_bit))
print(len(in_bit))
print(len(out_bit))
print(len(l_shift))
print(len(simd))
print(len(pe))

52
52
52
52
52
52


In [30]:
target_dir_hls_param = './param_hls/'
if not os.path.exists(target_dir_hls_param):
        os.makedirs(target_dir_hls_param)
        
hls_param_file = open(target_dir_hls_param + 'param.h', 'w')
hls_config_file = open(target_dir_hls_param + 'config.h', 'w')

config_file = open('config.json', 'r', encoding='utf-8')
config = json.load(config_file)
reader = QNNParamReader('mobilenetv2_Q_4w4a.npz')

In [31]:
 # conv_0 - 50
for i in range(51):
    processer = QNNLayerMemProcess('conv_' + str(i), reader, config, w_bit=w_bit[i], in_bit=in_bit[i], out_bit=out_bit[i], l_shift=l_shift[i], pe=pe[i], simd=simd[i])
    w, inc, bias = processer.conv()
    param_str = processer.layer_param_to_init_str(w, inc, bias)
    config_str = processer.conv_config_str()
    hls_param_file.write(param_str)
    hls_config_file.write(config_str)
    
# the last conv layer has no BN layer
processer = QNNLayerMemProcess('conv_' + str(51), reader, config, w_bit=w_bit[51], in_bit=in_bit[51],
                                   out_bit=out_bit[51], l_shift=l_shift[51], pe=pe[51], simd=simd[51])
w = processer.last_conv()
param_str = processer.last_layer_param_to_init_str(w)
config_str = processer.last_conv_config_str()
hls_param_file.write(param_str)
hls_config_file.write(config_str)

hls_param_file.close()
hls_config_file.close()

inc_q:  [ 1718  3217  3440  3388  6805 11248  4710  3036  1939  1629  1712  1260
  5252  1488  2435  2417  2805  5062  5475  5060  4090  2670  2049  1876
  1195  2084  1488  2937  5583  3927  4732  4829]
bias_q:  [ 3196797 -2766567  3428500 -3030807  -786042   901033 -2779695 -3381277
  3422814 -3472048 -3332624  3546237   235535  3249129  3136051  3142266
 -3161740  -255425 -1228256   116228  3094371 -2222455  3215699 -3301166
 -3405159  3142988 -3460788 -1467740  -915785 -2967526 -2587377  2649444]
inc_q:  [ 1858  5990  1176  2425  2830  7083  2412  2284  7946 10039  1385  1887
  2230  2967  6222  2848  2902  5622  4609  3422  7422  2645  3885  2891
  7835  4706  5918  5251  5396  1430  2036  1782]
bias_q:  [-112955  -82615  118252  103440   29649   -5992 -101489 -100250   85115
   -6282 -101943  122169 -107052  -95973   -8782  122856  -91200  -75841
   42659 -109784   49887  -87099 -102382 -100091  -37747  104357  -36337
  -90812  -19603   92598  -99322 -114387]
h mod simd != 0
inc_

inc_q:  [427 472 407 580 441 502 684 522 481 450 549 696 558 486 645 855 408 488
 503 670 416 409 478 464 636 483 338 392 285 393 331 371 533 530 353 443
 455 501 397 425 471 412 428 330 381 383 360 404 391 442 450 557 394 470
 636 439 278 461 393 420 285 354 385 424 331 336 389 321 548 347 502 424
 591 477 453 350 397 348 408 534 307 493 543 422 347 420 331 420 582 552
 407 522 375 589 529 519 489 425 338 297 473 508 514 426 451 530 480 509
 467 473 338 382 510 385 427 483 395 373 484 464 517 460 478 432 544 423
 327 472 473 363 525 518 298 613 400 426 550 566 467 352 388 374 293 373
 411 331 599 469 701 393 554 337 406 529 538 524 323 320 628 625 378 563
 608 485 486 452 467 540 472 441 510 471 582 598 529 562 343 523 434 497
 419 377 547 619 354 439 454 346 546 517 518 326]
bias_q:  [  2646   1696   2063   3516  -4912    604   7524  16482   3851   5202
  -2790  -1259   2660   1294  -2725  -3831  -3964  10915   3455    743
  -3950   2726  -1004   2749   4085   2428   1633   4155  -25

h mod simd != 0
inc_q:  [390 422 403 448 498 443 405 419 345 427 340 399 464 353 461 428 490 525
 377 528 436 467 424 458 526 512 540 393 419 415 457 501 357 423 521 385
 361 418 426 412 494 448 398 471 469 451 469 434 434 342 473 412 441 415
 385 339 438 401 451 353 420 430 418 478]
bias_q:  [ -87795  -61158    3399 -110352  -79316  107570  -43383   31079   13471
  133555   17508  -48132 -113029   19115  118430   80583  -70703  103516
 -135507 -140719  -77274   71853   -1523   45147   32234  118353 -168648
  -95176  115432   61691     869 -164675   89225  -73320 -117889 -112150
   32957   80918   28952  -34773 -140588  -97063   87106   12935  -94193
    7013  -35051   78801    8042   31960  220802  -62425   33838  -93534
   36649   92463   -9442  -68394  -81495   52061   -2757  -65761   81219
   -7488]
inc_q:  [361 345 344 419 380 355 290 303 275 286 424 324 322 276 340 398 310 251
 411 395 509 290 363 423 305 353 331 292 308 268 313 353 227 329 218 305
 383 410 309 349 479 375 374 27

inc_q:  [234 245 331 256 263 315 287 308 325 185 259 300 313 251 214 275 223 237
 425 333 291 317 267 341 229 292 327 248 213 249 258 352 422 282 240 236
 165 313 312 304 222 232 247 273 234 324 310 204 278 383 210 280 250 233
 288 306 358 256 311 185 296 266 420 289 250 206 277 281 338 269 271 383
 284 335 255 328 213 232 232 319 282 239 201 300 368 182 304 281 227 312
 230 282 324 266 222 247 426 251 242 261 314 333 312 182 196 261 276 224
 349 204 210 254 313 253 281 217 208 339 290 183 322 350 237 391 193 234
 353 281 230 211 318 342 268 333 318 299 256 272 300 271 296 301 236 208
 229 314 300 271 261 209 253 212 200 242 193 298 262 204 219 267 244 353
 382 286 233 264 237 375 200 328 250 317 179 283 189 268 196 277 326 227
 206 197 349 263 270 308 256 250 249 210 265 197 231 325 301 338 337 339
 222 215 278 274 206 225 359 213 337 279 362 226 287 264 258 346 204 339
 266 301 263 298 204 268 253 192 266 336 329 314 229 340 393 268 325 253
 303 282 224 369 268 389 249 206 214 220 28

inc_q:  [4492 2739 2677 2711 2292 2221 3668 2230 2058 2693 3509 2334 2900 2999
 3201 2723 5308 2647 2432 2526 2689 2490 2965 3329 2043 4489 3100 2708
 2525 3479 3850 2736 3575 2564 2821 3143 2789 4015 2791 3070 3218 3725
 1993 2646 3238 2947 2611 2700 1923 1818 3337 2012 2950 2001 3079 3156
 1895 3160 3598 2694 3450 3372 1953 3722 2647 1664 3639 1900 3190 3181
 2535 2409 2564 2847 3424 3149 2571 3380 2887 3104 3642 3363 3517 2875
 3049 3056 2846 3994 2774 2263 2855 1589 2686 3512 2294 4262 1896 2319
 3633 2720 2711 2767 3032 3469 2720 2296 4050 3465 3261 2561 2929 2859
 2041 2201 2403 2745 2973 3173 3119 3210 3245 2604 2631 3512 2655 3467
 3380 2066 3774 2383 3211 2891 3724 1917 3379 2746 2606 2877 2191 2802
 2766 1945 3388 1818 2214 3082 2246 3109 2367 1895 2325 3765 2406 3037
 2623 2149 1672 2871 3337 4327 3000 3704 3210 3728 2516 3113 2371 2777
 2897 2608 1848 3575 3204 2569 2978 2613 2442 1980 3605 4698 3181 4622
 4004 4167 2464 2224 3092 2160 3000 3064 2790 2406 3398 3134 2810 353

...

inc_q:  [2163 2579 2086 2077 3386 3236 1780 1565 1965 2220 1942 3037 2582 2453
 1855 2456 1948 2354 2260 3280 2931 2960 2132 2731 2141 2912 1991 2505
 2408 1758 3856 2958 2824 2442 2311 2245 1595 2571 2315 2487 1968 1954
 2335 2469 2030 3047 2548 2315 2395 2631 2547 2335 3039 2425 1910 2040
 1853 2406 2065 2418 2497 2209 2740 2702 2151 2609 2508 2444 1827 1960
 1981 2323 2983 2784 2589 2065 2151 2615 2756 2601 2577 2226 2002 2550
 2175 2857 2726 2763 2095 2145 2570 2918 2590 2327 2431 2506 2179 2728
 3335 2649 1854 2251 2112 2750 1728 2630 2620 2310 2337 2526 2481 2416
 2685 2434 2689 2852 2967 1754 2617 2092 2063 2525 2187 3331 1726 2127
 2331 2035 2944 2259 3008 2779 2070 2264 2307 2492 2388 2346 2846 2767
 2510 2905 2731 2426 2077 3478 3250 1614 2305 2645 2932 2015 3061 2896
 2102 2429 2161 2377 1749 1865 1777 1806 2426 2233 2090 2667 2421 2013
 2071 2165 2456 1914 1661 2207 1912 2242 2232 2241 2850 3017 2759 2103
 2580 1804 2960 2400 1761 2421 2980 2036 2523 2635 2915 2367 2892 331

inc_q:  [148 137 130 116 141  99 139 130 123 176 147 119 105 127 149 132 144 196
 115 129 115 134 139 125  99 156 124 144 124 125 123 154 124 110  99 111
 132 126  93 109 127 145 122 148 116 126 117 108 102 139  89 173 106 124
 139 117 122 119 132  86  88  81 125 149 108 116 102 124 122 137 130 104
 123 131 140 137 136 111 136 132 179 117 164 110 131 123 119 143 128 136
 124 107 147 132 131 129 141 156 105 127 118 117 148 136 102 104 123 123
 121 129 113 147 101 130 114 130 116 107 134 110 106 110 111 150 115 104
 142 111 146 158 109 131 115 118 146  85 113 111 151 130 122 136 162 124
 115 104 137 124 120 134 173 116 134 130 138 126 109 125 134 111 124 156
 106 140 161  97 139 131 108 124 121 132 146 146 129 124 113 138 112 151
 128 112 124 155 127 183 124 121  99 142 136 146 101 130 132 133 117 117
 148 108 116 138 106 132 120 103 152 105  92 153 158 152 112 127 145 129
 130 125 124 124 118 113 100 122 124 113 111 131 108 146 118 121 103 126
 146 117 162 109 114 124 153 119 102 101 13

inc_q:  [2060 2212 2087 2538 2385 2156 2166 2036 2075 2325 2512 2494 2014 2734
 1876 2505 2671 1718 1950 2817 2196 2256 2305 1428 1843 2308 2394 2277
 2454 2380 2934 1635 2100 2125 2301 2859 2380 2266 1903 2093 1918 1786
 2689 2294 2165 2047 1864 2201 1871 2389 2636 1825 2440 2169 2642 2892
 2268 1973 2956 2277 2481 1948 2605 2283 2153 2280 2321 2023 2113 2184
 2812 2520 2943 1721 2737 2331 2179 2420 2855 2149 2318 1854 2675 2098
 1655 2461 2463 2232 2187 2579 1734 2482 2557 1878 2848 2840 2388 2336
 2506 2160 1633 2350 2194 2261 2604 2218 1858 1957 1752 1920 2108 2354
 2000 1889 2025 2320 2252 2408 2203 2291 1887 2773 1781 2190 2236 2411
 1908 1912 1982 1803 3233 1963 2721 2336 2511 3338 2224 2348 2099 2161
 2298 2025 2212 2469 3042 2455 2081 2628 2546 2753 2343 2107 2733 2587
 2033 3713 2995 2379 2411 1321 2091 2262 3001 1861 2708 2246 2249 2048
 3205 2258 2259 2297 2240 2316 2644 1687 2096 2286 1731 2573 2408 2317
 1877 2280 1797 1945 2171 1577 2677 2357 2442 2104 2229 2369 2009 186

inc_q:  [ 75  94 105  86  93  92  87  79  79 103 110  84  85  98 102  88  65  88
  95  81  91  75 111  86  90 100  76  70  78  88  89 111  85  90  81  93
  71  82  84  82  82  97 102 104  81  73 105 100  84  90 115 102  91  93
 103  85 104  82  76  82  86  82  98 112  94 100  98  78  76  95  83  81
  76  94  80  80  73  85  66  80  91 109  85  94 103  97  84  83 126  98
  76  81  91  84  98  88  78  76  87  88  78  82  87  95  86  93  93  68
  82  78  76  73  80  81  85  79 105  78  69 102  89 103 113  81  80  91
  86  81  89  94  88  85 120  92  90  86 102  92  87  69  91  88  83  80
  86  79  87 109  86  85  70  77  92  87  96  98  82  73  84  96 102  78
  83 103  92  96  87  80  86  91  83  93  76  93  93  79  94  92  94  76
  90  93  80  92  83  86  89  89  99  86  88  84  98  85  69  84  85  68
  81  75  86  78  77  87  93  97  97 112  82  84  91  94 100  74  82  92
  86 100  79  95  74 102  68  84  82  87 104  81  73  84  94  91  78  85
  98  92  94  84  66  96  97  73  97  91  7

inc_q:  [1460 1882 1577 2140 1652 1675 1772 1549 2284 1749 1327 2348 2059 1793
 1734 2084 1511 1631 1678 1274 1916 2069 1432 1500 1572 1977 2138 1568
 2671 1951 2119 1701 1382 1667 1569 1567 1952 1478 2274 1793 1475 1694
 1350 1201 2060 2366 2011 1549 1715 1748 1163 1630 1445 2440 2296 2308
 1408 1434 1794 2428 2029 2042 1402 1665 2038 2002 1699 1611 1998 2067
 1360 2120 1464 1517 1938 1516 1512 1302 1851 2108 1361 1573 1978 1507
 1691 1528 1440 1989 1676 1659 1824 1650 1922 1766 1832 2095 2236 1576
 1776 1579 2162 1607 1506 1285 1926 1423 1986 1698 2045 1559 2376 2370
 1650 2544 1405 1518 1571 1976 3139 1668 2088 1607 1807 1897 1829 1564
 2120 1907 1370 1698 1829 1438 1448 2070 1899 2631 1828 1680 2514 2621
 1855 1779 2059 1603 1697 1796 1634 1683 2290 1320 2236 1794 1842 1635
 2426 1830 1442 1601 1990 1757 1808 1883 1230 1340 2018 1563 1854 1733
 1828 1523 1822 1489 1171 1400 1569 1808 2222 1514 1792 1929 1845 1869
 1580 1454 1815 2270 1488 2090 1797 1940 1691 1523 1411 1891 2053 157

In [35]:
param_dict = np.load("mobilenetv2_Lite_0.25_Q_4w4a.npz")
param_dict